# Atividade 3 - parte 1
Alunas: Bianca Bonetti Nunes e Bianca Suganuma Cadamura

### Criando valores aleatórios para a radiação, cobertura de nuvens e velocidade do vento


In [51]:
import pandas as pd
import numpy as np

# Gerar um intervalo de datas para o ano de 2023
datas = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

# Criar DataFrame com as datas
df = pd.DataFrame({'Data': datas})

# Extrair os meses das datas e adicioná-los como uma nova coluna no DataFrame
df['Mes'] = df['Data'].dt.month

# Extrair o dia do ano das datas e adicioná-lo como uma nova coluna no DataFrame
df['Dia_do_Ano'] = df['Data'].dt.dayofyear

# Expandir as datas para incluir as horas do dia (1 a 24)
df_expandido = df.loc[df.index.repeat(24)].reset_index(drop=True)
df_expandido['Hora_Dia'] = [hora for _ in range(len(df)) for hora in range(1, 25)]

In [52]:
# Parâmetros de aleatoriedade para radiação, cobertura de nuvens e velocidade do vento
radMax = 1000
radMin = 1

cobertura_de_nuvensMax = 100
cobertura_de_nuvensMin = 0

veloc_ventoMax = 11.31
veloc_ventoMin = 0

# Número de linhas/número de datas
n = df_expandido.shape[0]

# Criar valores aleatórios
radiacao = (radMax - radMin) * np.random.rand(n) + radMin
cobertura_de_nuvens = (cobertura_de_nuvensMax - cobertura_de_nuvensMin) * np.random.rand(n) + cobertura_de_nuvensMin
veloc_vento = (veloc_ventoMax - veloc_ventoMin) * np.random.rand(n) + veloc_ventoMin

# Adicionar as colunas ao DataFrame
df_expandido['Radiacao'] = radiacao
df_expandido['Cobertura_de_Nuvens'] = cobertura_de_nuvens
df_expandido['Veloc_Vento'] = veloc_vento


In [53]:
df_expandido

,Data,Mes,Dia_do_Ano,Hora_Dia,Radiacao,Cobertura_de_Nuvens,Veloc_Vento
0,2023-01-01,1,1,1,252.451620,37.491283,2.296892
1,2023-01-01,1,1,2,829.981866,1.535256,0.231764
2,2023-01-01,1,1,3,276.984547,95.757676,8.446963
3,2023-01-01,1,1,4,395.132286,81.844974,11.228883
4,2023-01-01,1,1,5,390.157012,49.847280,3.374589
...,...,...,...,...,...,...,...
8755,2023-12-31,12,365,20,578.193695,35.956736,10.768172
8756,2023-12-31,12,365,21,928.961141,29.493603,9.928893
8757,2023-12-31,12,365,22,336.013119,62.253623,4.159139
8758,2023-12-31,12,365,23,896.866237,59.062658,3.836732


### 1. Implementar uma função para determinar a classe de estabilidade de Pasquil para diferentes condições atmosféricas

In [54]:
# Função para determinar a classe de estabilidade
def classe_estabilidade(hora_dia, radiacao, cobertura_de_nuvens, veloc_vento):
    if hora_dia < 6 or hora_dia >= 18:
        # Condições noturnas
        if cobertura_de_nuvens >= 5:
            if 2 <= veloc_vento <= 3:
                return "E"
            else:
                return "D"
        else:
            if 2 <= veloc_vento < 3:
                return "F"
            elif 3 <= veloc_vento <= 5:
                return "E"
            else:
                return "D"
    else:
        # Condições diurnas
        if radiacao > 700:  # Radiação solar forte
            if veloc_vento < 2:
                return "A"
            elif 2 <= veloc_vento <= 5:
                return "B"
            else:
                return "C"
        elif 350 <= radiacao <= 700:  # Radiação solar moderada
            if veloc_vento < 3:
                return "B"
            elif 3 < veloc_vento <= 5:
                return "C"
            else:
                return "D"
        else:  # Radiação solar fraca
            if veloc_vento < 2:
                return "B"
            elif 2 <= veloc_vento <= 3:
                return "C"
            elif 3 < veloc_vento <= 5:
                return "C"
            elif 5 < veloc_vento <= 6:
                return "C"
            else:
                return "D"

# Aplicar a função para cada linha do DataFrame
df_expandido['Classe_Estabilidade'] = df_expandido.apply(
    lambda row: classe_estabilidade(
        row['Hora_Dia'],
        row['Radiacao'],
        row['Cobertura_de_Nuvens'],
        row['Veloc_Vento']
    ), axis=1
)

# Verificar o DataFrame expandido
print(df_expandido)


           Data  Mes  Dia_do_Ano  Hora_Dia    Radiacao  Cobertura_de_Nuvens  \
0    2023-01-01    1           1         1  252.451620            37.491283   
1    2023-01-01    1           1         2  829.981866             1.535256   
2    2023-01-01    1           1         3  276.984547            95.757676   
3    2023-01-01    1           1         4  395.132286            81.844974   
4    2023-01-01    1           1         5  390.157012            49.847280   
...         ...  ...         ...       ...         ...                  ...   
8755 2023-12-31   12         365        20  578.193695            35.956736   
8756 2023-12-31   12         365        21  928.961141            29.493603   
8757 2023-12-31   12         365        22  336.013119            62.253623   
8758 2023-12-31   12         365        23  896.866237            59.062658   
8759 2023-12-31   12         365        24  392.752664            98.648528   

      Veloc_Vento Classe_Estabilidade  
0        2.

### 2. Implementar a função de estimativa de coefiente de dispersão (sigmaYZ) para todas as classes de estabilidade

In [55]:
# Cria uma lista de valores de 0 a 50000 em incrementos de 1000
x_values = list(np.arange(0, 50001, 1000))
x_values[0] = 1
# Função da estimativa da dispersão lateral e vertical da pluma
def sigmaYZ(classe, x, urbOrRural):
    if urbOrRural == 'urbano':
        if classe in ['A', 'B']:
            sigmaY = 0.32 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.24 * x * (1 + 0.001 * x) ** (0.5)
        elif classe == 'C':
            sigmaY = 0.22 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.20 * x
        elif classe == 'D':
            sigmaY = 0.16 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.14 * x * (1 + 0.0003 * x) ** (-0.5)
        elif classe in ['E', 'F']:
            sigmaY = 0.11 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.08 * x * (1 + 0.0015 * x) ** (-0.5)
        else:
            raise ValueError('Classe de estabilidade errada')
    elif urbOrRural == 'rural':
        if classe == 'A':
            sigmaY = 0.22 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.20 * x
        elif classe == 'B':
            sigmaY = 0.16 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.12 * x
        elif classe == 'C':
            sigmaY = 0.11 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.08 * x * (1 + 0.0002 * x) ** (-0.5)
        elif classe == 'D':
            sigmaY = 0.08 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.06 * x * (1 + 0.0015 * x) ** (-0.5)
        elif classe == 'E':
            sigmaY = 0.06 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.03 * x * (1 + 0.0003 * x) ** (-1)
        elif classe == 'F':
            sigmaY = 0.04 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.016 * x * (1 + 0.0003 * x) ** (-1)
        else:
            raise ValueError('Classe de estabilidade errada. Fique ligado')
    else:
        raise ValueError('Tipo de área errada')
    return sigmaY, sigmaZ

# Define o tipo de área
urbOrRural = 'urbano'
sY = np.zeros((df_expandido.Classe_Estabilidade.shape[0],len(x_values)))
sZ = np.zeros((df_expandido.Classe_Estabilidade.shape[0],len(x_values)))
for ii in range(0,df_expandido.Classe_Estabilidade.shape[0]):
    for jj in range (0, len(x_values)):
        sigmaY,sigmaZ = sigmaYZ(df_expandido.Classe_Estabilidade[ii],x_values[jj],urbOrRural)
        sY[ii,jj]=sigmaY
        sZ[ii,jj]=sigmaZ


In [56]:
# Geração das colunas sigmaY e sigmaZ
for x in x_values:
    sigmaY = []
    sigmaZ = []
    for i in range(df_expandido.shape[0]):
        sigmaY_i, sigmaZ_i = sigmaYZ(df_expandido.loc[i, 'Classe_Estabilidade'], x, urbOrRural)
        sigmaY.append(sigmaY_i)
        sigmaZ.append(sigmaZ_i)
    df_expandido[f'sigmaY_{x}'] = sigmaY
    df_expandido[f'sigmaZ_{x}'] = sigmaZ

C:\Users\BiaBN\AppData\Local\Temp\ipykernel_7960\2617540073.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_expandido[f'sigmaZ_{x}'] = sigmaZ
C:\Users\BiaBN\AppData\Local\Temp\ipykernel_7960\2617540073.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_expandido[f'sigmaY_{x}'] = sigmaY
C:\Users\BiaBN\AppData\Local\Temp\ipykernel_7960\2617540073.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 

In [57]:
print(df_expandido)
#Salvar o DataFrame em um arquivo CSV
df_expandido.to_csv(r'C:\Users\BiaBN\OneDrive\Documentos\controledepoluicaoatm\tabelas\estabilidade.csv')

           Data  Mes  Dia_do_Ano  Hora_Dia    Radiacao  Cobertura_de_Nuvens  \
0    2023-01-01    1           1         1  252.451620            37.491283   
1    2023-01-01    1           1         2  829.981866             1.535256   
2    2023-01-01    1           1         3  276.984547            95.757676   
3    2023-01-01    1           1         4  395.132286            81.844974   
4    2023-01-01    1           1         5  390.157012            49.847280   
...         ...  ...         ...       ...         ...                  ...   
8755 2023-12-31   12         365        20  578.193695            35.956736   
8756 2023-12-31   12         365        21  928.961141            29.493603   
8757 2023-12-31   12         365        22  336.013119            62.253623   
8758 2023-12-31   12         365        23  896.866237            59.062658   
8759 2023-12-31   12         365        24  392.752664            98.648528   

      Veloc_Vento Classe_Estabilidade  sigmaY_1  si

### 3. Implementar a função de estimativa de sobrelevação da pluma utilizando os métodos de Davidson-Bryant, Holland e Briggs. Deve ser considerado o efeito Tip-Downwash

In [58]:
# Função para estimar o deltaH com base na equação de Davidson-Bryant
def deltaHdavidsonBryant(d,vs,u,Ts,Tamb):
    deltaH = (d*(vs/u)**(1.4))*(1+(Ts-Tamb)/Ts)
    return deltaH

In [59]:
# Função para estimar o deltaH com base na equação de Holland
def deltaHHolland(d,vs,u,p,Ts,Tamb):
    deltaH = ((vs*d/u)*(1.5+2.68*10^(-3)*p*(Ts-Tamb)*d/Ts))
    return deltaH

In [60]:
# Função para estimar o deltaH com base na equação de Briggs
def deltaHBriggs(d,vs,Ts,Tamb,classe,u):
    Fb = 9.8*((d/2)**2)*vs*(1-Tamb/Ts)
    if classe == 'A':
        gradT = -0.02 + 0.0098
    elif classe == 'B':
        gradT = -0.018 + 0.0098 
    elif classe == 'C':
        gradT = -0.016 + 0.0098 
    elif classe == 'D':
        gradT = -0.01 + 0.0098 
    elif classe == 'E':
        gradT = 0.005 + 0.0098 
    elif classe == 'F':
        gradT = 0.028 + 0.0098 
    else:
        print('Classe de estabilidade errada')
    s = (9.8/Tamb)*(gradT)
    if classe == 'A' or classe == 'B' or classe == 'C' or classe == 'D':
        if Fb < 55:
            deltaTc = 0.0297*Ts*(vs**(1/3))/(d**(2/3))
            if (Ts - Tamb) < deltaTc:
                deltaH = 3*d*vs/u
            else:
                deltaH = 21.425*Fb**(3/4)/u
        else:
            deltaTc = 0.0057*Ts*(vs**(2/3)/d**(1/3))
            if (Ts - Tamb) < deltaTc:
                deltaH = 3*d*vs/u
            else:
                deltaH = 2.6*(Fb/(u*s))**(1/3)
    elif classe == 'E' or classe == 'F':
        deltaTc = 0.019582*Ts*vs*s**(1/2)
        if (Ts - Tamb) < deltaTc:
            Fm = (vs**2)*(d**2)*Tamb/(4*Ts) 
            deltaH1 = 1.5*(Fm/(u*s**(1/2)))**(1/3)
            deltaH2 = 3*d*vs/u
            if deltaH1 > deltaH2:
                deltaH = deltaH2
            else:
                deltaH = deltaH1
        else:
            deltaH = 2.6*(Fb/(u*s))**(1/3)
    return deltaH

### Utilizando o modelo de Briggs para calcular deltaH

In [61]:
d = 1 #diametro da chaminé
vs = 20 #velociadade na saida da chaminé
Ts = 400 # em Kelvin
Tamb = 293 # em Kelvin

dH = np.zeros((df_expandido.Classe_Estabilidade.shape[0]))
for ii in range(0,df_expandido.Classe_Estabilidade.shape[0]):
    deltaH = deltaHBriggs(d,vs,Ts,Tamb,df_expandido.Classe_Estabilidade[ii],df_expandido.Veloc_Vento[ii])
    dH[ii] = deltaH
df_expandido['deltaH'] = dH
print(df_expandido)

           Data  Mes  Dia_do_Ano  Hora_Dia    Radiacao  Cobertura_de_Nuvens  \
0    2023-01-01    1           1         1  252.451620            37.491283   
1    2023-01-01    1           1         2  829.981866             1.535256   
2    2023-01-01    1           1         3  276.984547            95.757676   
3    2023-01-01    1           1         4  395.132286            81.844974   
4    2023-01-01    1           1         5  390.157012            49.847280   
...         ...  ...         ...       ...         ...                  ...   
8755 2023-12-31   12         365        20  578.193695            35.956736   
8756 2023-12-31   12         365        21  928.961141            29.493603   
8757 2023-12-31   12         365        22  336.013119            62.253623   
8758 2023-12-31   12         365        23  896.866237            59.062658   
8759 2023-12-31   12         365        24  392.752664            98.648528   

      Veloc_Vento Classe_Estabilidade  sigmaY_1  si

C:\Users\BiaBN\AppData\Local\Temp\ipykernel_7960\1391082514.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_expandido['deltaH'] = dH


### 4. Implementar a função do modelo gaussiano

In [62]:
# Criando uma função do modelo gaussiano
import numpy as np
def modeloGaussiano(qs,sigmaY,sigmaZ,u,y,z,H):
    termo1 = qs/(2*np.pi*sigmaY*sigmaZ*u)
    termo2 = np.exp((-y**2)/(2*sigmaY**2))
    termo3 = np.exp((-(z-H)**2)/(2*sigmaZ**2)) + np.exp((-(z+H)**2)/(2*sigmaZ**2))
    conc = termo1*termo2*termo3
    conc = conc*10**6
    return conc

### 5. Realizar simulações com o script criado, utilizando diferentes classes de estabilidade, velocidades do vento, alturas de chaminé. Considere a taxa de emissão que você estimou na primeira atividade. Encontre a altura de chaminé necessária para que as concentrações não violem os padrões da Resolução CONAMA 491.

In [63]:
# Estimando a sobrelevação da pluma (deltaH)
hg = 100 #altura da chaminé
Hefetiva = np.zeros((df_expandido.Classe_Estabilidade.shape[0]))

for ii in range(0,df_expandido.Classe_Estabilidade.shape[0]):
    if vs>1.5*(df_expandido['Veloc_Vento'][ii]):
        Hefetiva[ii] = hg+(df_expandido['deltaH'][ii])
    else:
        Hefetiva[ii] = hg +2*d*((vs/u)-1.5)
df_expandido['Hefetiva'] = Hefetiva
print(df_expandido)

           Data  Mes  Dia_do_Ano  Hora_Dia    Radiacao  Cobertura_de_Nuvens  \
0    2023-01-01    1           1         1  252.451620            37.491283   
1    2023-01-01    1           1         2  829.981866             1.535256   
2    2023-01-01    1           1         3  276.984547            95.757676   
3    2023-01-01    1           1         4  395.132286            81.844974   
4    2023-01-01    1           1         5  390.157012            49.847280   
...         ...  ...         ...       ...         ...                  ...   
8755 2023-12-31   12         365        20  578.193695            35.956736   
8756 2023-12-31   12         365        21  928.961141            29.493603   
8757 2023-12-31   12         365        22  336.013119            62.253623   
8758 2023-12-31   12         365        23  896.866237            59.062658   
8759 2023-12-31   12         365        24  392.752664            98.648528   

      Veloc_Vento Classe_Estabilidade  sigmaY_1  si

C:\Users\BiaBN\AppData\Local\Temp\ipykernel_7960\2895479443.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_expandido['Hefetiva'] = Hefetiva


In [66]:
# Parâmetros do modelo, utilizando como qs a emissão máxima (calculada na atividade 1) de cada poluente
qsMP = 320654.82  # g/s
qsSO2 = 547.19    # g/s
qsCO = 12147.67   # g/s
y = 0             # estimando na direção do vento
z = 1.5           # altura do nariz
H = 50            # m
urbOrRural = 'urbano'

# Inicializa listas para concentrações
concMP = np.zeros((df_expandido.shape[0], len(x_values)))
concSO2 = np.zeros((df_expandido.shape[0], len(x_values)))
concCO = np.zeros((df_expandido.shape[0], len(x_values)))

# Calcular concentrações para cada linha do DataFrame
for i in range(df_expandido.shape[0]):
    u = df_expandido.loc[i, 'Veloc_Vento']
    H_efetiva = df_expandido.loc[i, 'Hefetiva']
    
    for j, x in enumerate(x_values):
        sigmaY = df_expandido.loc[i, f'sigmaY_{x}']
        sigmaZ = df_expandido.loc[i, f'sigmaZ_{x}']
        
        concMP[i, j] = modeloGaussiano(qsMP, sigmaY, sigmaZ, u, y, z, H_efetiva)
        concSO2[i, j] = modeloGaussiano(qsSO2, sigmaY, sigmaZ, u, y, z, H_efetiva)
        concCO[i, j] = modeloGaussiano(qsCO, sigmaY, sigmaZ, u, y, z, H_efetiva)

# Adiciona colunas de concentrações ao DataFrame para todas as distâncias x_values
for j, x in enumerate(x_values):
    df_expandido[f'concMP_{x}'] = concMP[:, j]
    df_expandido[f'concSO2_{x}'] = concSO2[:, j]
    df_expandido[f'concCO_{x}'] = concCO[:, j]

# Verifica o DataFrame expandido com concentrações
print(df_expandido)


           Data  Mes  Dia_do_Ano  Hora_Dia    Radiacao  Cobertura_de_Nuvens  \
0    2023-01-01    1           1         1  252.451620            37.491283   
1    2023-01-01    1           1         2  829.981866             1.535256   
2    2023-01-01    1           1         3  276.984547            95.757676   
3    2023-01-01    1           1         4  395.132286            81.844974   
4    2023-01-01    1           1         5  390.157012            49.847280   
...         ...  ...         ...       ...         ...                  ...   
8755 2023-12-31   12         365        20  578.193695            35.956736   
8756 2023-12-31   12         365        21  928.961141            29.493603   
8757 2023-12-31   12         365        22  336.013119            62.253623   
8758 2023-12-31   12         365        23  896.866237            59.062658   
8759 2023-12-31   12         365        24  392.752664            98.648528   

      Veloc_Vento Classe_Estabilidade  sigmaY_1  si

In [79]:
# Encontrar a maior concentração para cada classe de estabilidade e para cada poluente
max_concentrations = df_expandido.groupby('Classe_Estabilidade').agg({
    **{f'concMP_{x}': 'max' for x in x_values},
    **{f'concSO2_{x}': 'max' for x in x_values},
    **{f'concCO_{x}': 'max' for x in x_values}
})

# Calcular a maior concentração de cada poluente para cada classe
max_conc_MP = max_concentrations[[f'concMP_{x}' for x in x_values]].max(axis=1)
max_conc_SO2 = max_concentrations[[f'concSO2_{x}' for x in x_values]].max(axis=1)
max_conc_CO = max_concentrations[[f'concCO_{x}' for x in x_values]].max(axis=1)

max_conc_df = pd.DataFrame({
    'Classe_Estabilidade': max_conc_MP.index,
    'Max_Conc_MP': max_conc_MP.values,
    'Max_Conc_SO2': max_conc_SO2.values,
    'Max_Conc_CO': max_conc_CO.values
})

# Verificar as maiores concentrações
print(max_conc_df)


# Função para localizar os valores no DataFrame
def localizar_valores(df, valores):
    for valor in valores:
        # Localizar todas as ocorrências do valor
        result = (df == valor).stack()
        
        # Filtrar apenas os True
        locations = result[result]
        
        # Exibir as linhas e colunas onde o valor foi encontrado
        for (index, col), is_found in locations.items():
            print(f"Valor {valor} encontrado na linha {index}, coluna '{col}'")

# Chamar a função
print('Localização no dataframe original dos valores maximos de MP para cada classe (respectivamente)')
localizar_valores(df_expandido, max_conc_MP.values) 
print(' ')
print('Localização no dataframe original dos valores maximos de SO2 para cada classe (respectivamente)')
localizar_valores(df_expandido, max_conc_SO2.values) 
print(' ')
print('Localização no dataframe original dos valores maximos de CO para cada classe (respectivamente)')
localizar_valores(df_expandido, max_conc_CO.values) 

  Classe_Estabilidade   Max_Conc_MP  Max_Conc_SO2   Max_Conc_CO
0                   A  1.129130e+06   1926.833278  42775.881891
1                   B  1.129152e+06   1926.871734  42776.735605
2                   C  9.402852e+05   1604.574850  35621.714158
3                   D  1.138704e+06   1943.172245  43138.608507
4                   E  6.451836e+05   1100.990893  24442.102457
5                   F  7.915526e+05   1350.766191  29987.137803
Localização no dataframe original dos valores maximos de MP para cada classe (respectivamente)
Valor 1129129.5127488335 encontrado na linha 8360, coluna 'concMP_1000'
Valor 1129152.0477284421 encontrado na linha 5412, coluna 'concMP_1000'
Valor 940285.2021374193 encontrado na linha 447, coluna 'concMP_1000'
Valor 1138704.1914948118 encontrado na linha 3913, coluna 'concMP_1000'
Valor 645183.6412933285 encontrado na linha 6378, coluna 'concMP_3000'
Valor 791552.6413407701 encontrado na linha 2303, coluna 'concMP_3000'
 
Localização no dataframe or

In [ ]:
df_expandido

In [ ]:
# Criando dominio de modelagem
x = np.linspace(-100,10000,500)
# print(x)
y = np.linspace(-10000,10000,500)

# Criando matrizes de x e y
xx,yy = np.meshgrid(x,y)
#print(xx.shape)


# Adotando inputs
classe = 'F'
urbOrRural = 'urbano' 
hg = 150 # m altura geométrica da chaminé
qs = 100 # g/s
sigmaY = 100 # m
sigmaZ = 150 # m
u = 5 # m/s
y = 0 # estimando na direção do vento
z = 1.5 # altura do nariz
d = 1 # em metros
vs = 10 # em m/s
Ts = 300 # em Kelvin
Tamb = 293 # em Kelvin

# Estimando o coeficiente de dispersão lateral e vertical
#sigmaY,sigmaZ = sigmaXY(xx,classe,urbOrRural)
#print(sigmaY)

if vs>1.5*u:
    # Estimando a sobrelevação da pluma (deltaH)
    deltaH = deltaHdavidsonBryant(d,vs,u,Ts,Tamb)
    print('deltaH = '+str(deltaH))
    hef = hg+deltaH
    print('A pluma subiu')
else:
    hef = hg +2*d*((vs/u)-1.5)
    print('A pluma caiu')

# Utilizando a matriz de yy como input na função do modelo gaussiano
conc = modeloGaussiano(qs,sigmaY,sigmaZ,u,yy,z,H)

# Visualização dos resultados no espaço
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
fig,ax = plt.subplots()
ax.contourf(xx,yy,conc+0.1,norm = LogNorm())

# Corte em y - sobre o eixo x
fig,ax = plt.subplots()
ax.plot(x,conc[250,:])

# Corte em x - sobre o eixo y 
fig,ax = plt.subplots()
ax.plot(x,conc[:,100])